In [1]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore
!pip install Flask-Mail

  Using cached https://files.pythonhosted.org/packages/b8/6c/bedcab39e8dc969f7a48d058dbacd69fc07ce3f817a03de875902016f667/google_cloud_datastore-1.15.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/be/66b4b7c7e94879257f404a716385c62c0629e011351009c646dd04813175/google_cloud_core-1.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
google-resumable-media 0.7.1 requires google-crc32c<0.2dev,>=0.1.0; python_version >= "3.5", which is not installed.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.2 which is incompatible.
tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.1 which is incompatible.
google-cloud-bigquery 1.9.0 has requirement google-cloud-core<0.30dev,>=0.29.0, but you'll have google-cloud-core 1.4.4 which is incompatible.
  Found existing installation: six 1.1

In [3]:

import json
import re
from flask import Flask, request, jsonify, make_response
from flask import render_template
from google.cloud import datastore

In [6]:
from send_invitation import send_email 
from threading import Thread
import jwt

app = Flask(__name__)
results = ""
rsize = 0

import re
from random import seed
from random import randint
from datetime import datetime

seed(datetime.now())

app.config['SECRET_KEY'] = 'SECRET_KEY_CHAT9900'

def get_public_url():
    '''
    tunnel = !curl https://4040-44eb2645-47f4-496a-9df1-70647bdfe449.cs-asia-southeast1-vjqr.cloudshell.dev/api/tunnels
    tunnel= str(tunnel)
    mobj = re.search('public_url', tunnel)
    i = mobj.span()
    t = tunnel[i[1]:].split('\"')
    public_url =t[2]
    '''
    # populate this from your local server
    public_url = "http://287bab297532.ngrok.io"
    return public_url

def gen_jwt():
    return jwt.encode(
            {'chatroom': 101 },
            app.config['SECRET_KEY'], algorithm='HS256')

@app.route('/chatroom/<token>', methods=['GET', 'POST'])
def reset_password(token):
    meeting = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['chatroom']
    print("meeting click was clicked ",meeting)
    return render_template('place_holder.html')

def send_password_reset_email():
    #user should have username and email
    user = "Test User"
    token = jwt.encode(
            {'reset_password': 101 },
            app.config['SECRET_KEY'], algorithm='HS256')

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    topic = req.get('queryResult').get('parameters').get('topic')
    intent = req.get('queryResult').get('intent').get('displayName')

    print(topic)
    # there could be multiple topics in a sentence
    if isinstance(topic, list):
        topic = topic[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    topic = topic.lower()
    print("topic is ",topic)
    
    rsp = buildReply('Command not recognized')
    if (topic == "schedule"):
        rsp = scheduleMovie(req)
    elif (topic == "profile" or topic == "account"):
        rsp = getProfileAccount()
    elif (topic == "movie recommendation" ):
        rsp = getRecommendations()
        
    rsp = json.dumps(rsp, indent=4)
    print (rsp)
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def scheduleMovie(req):
    confirm = req.get('queryResult').get('parameters').get('confirm').lower()
    print(confirm)
    if ("yes" in confirm):
        attendees = ["chatbot9900@gmail.com", "e.life.hacker@gmail.com"]
        public_url = get_public_url()
        token = gen_jwt()
        chatroom_url = public_url+"/"+token.decode("utf-8")
        Thread(target=send_email, args=(attendees,chatroom_url,)).start()
        return buildReply('Sending request ...')
    return buildReply('Aborted')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print (urls)
    return buildReply(urls)

def getRecommendations():
    global results 
    global rsize
    print ("recom")
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print(rsize)
    return buildReply(results[randint(0, rsize -1 )]['title']+"\n"+results[randint(0, rsize-1)]['title'])

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Mar/2021 14:25:25] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "originalDetectIntentRequest": {
        "payload": {}
    },
    "session": "projects/i-backbone-307006/agent/sessions/dfMessenger-54702431",
    "queryResult": {
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.6,
                "score": 0.6
            }
        },
        "parameters": {
            "confirm": "yes",
            "topic": "schedule"
        },
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ],
        "intentDetectionConfidence": 1.0,
        "queryText": "yes",
        "outputContexts": [
            {
                "parameters": {
                    "confirm": "yes",
                    "friend.original": "James",
                    "movie": "The lord of the ring",
                    "time": "2021-03-21T18:00:00+11:00",
                    "movie.o